In [33]:
import pandas as pd
import numpy as np
import os

In [26]:
os.getcwd()

'D:\\workspace\\DACON_2019'

In [27]:
os.chdir('D:\\workspace\\DACON_2019')

In [28]:
test = pd.read_csv("./SOOMIN/data/Test_데이콘전처리및시간별평균.csv")
test.head()

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
0,2017-07-01 00:00:00,1.187315,0.351692,0.264282,0.253713,0.176311,0.146826,0.307492,0.229566,0.061527,...,0.260910,0.317195,0.321455,0.367367,0.159644,0.204865,0.153908,0.275,0.021,0.303634
1,2017-07-01 01:00:00,1.186123,0.462480,0.287727,0.313939,0.184959,0.161964,0.394632,0.278938,0.071040,...,0.246349,0.289677,0.279606,0.295989,0.159288,0.246143,0.172024,0.222,0.021,0.323445
2,2017-07-01 02:00:00,1.168301,0.439076,0.295718,0.305454,0.187243,0.155218,0.374071,0.283294,0.073194,...,0.218539,0.236890,0.268077,0.243984,0.157027,0.247709,0.157602,0.237,0.021,0.326541
3,2017-07-01 03:00:00,1.163425,0.393705,0.292820,0.303389,0.181969,0.153222,0.352247,0.280866,0.071465,...,0.195375,0.198678,0.247191,0.206447,0.154466,0.239162,0.146758,0.229,0.020,0.320604
4,2017-07-01 04:00:00,1.398466,0.370926,0.295090,0.308605,0.181614,0.150549,0.336359,0.290058,0.073235,...,0.174982,0.173972,0.245303,0.216265,0.152014,0.234301,0.141317,0.202,0.021,0.336269


In [29]:
test['Time'] = pd.to_datetime(test['Time'])
test = test.set_index('Time')

In [30]:
test.iloc[:, 0]

Time
2017-07-01 00:00:00    1.187315
2017-07-01 01:00:00    1.186123
2017-07-01 02:00:00    1.168301
2017-07-01 03:00:00    1.163425
2017-07-01 04:00:00    1.398466
                         ...   
2018-06-30 19:00:00    1.255000
2018-06-30 20:00:00    1.596000
2018-06-30 21:00:00    1.379000
2018-06-30 22:00:00    1.000000
2018-06-30 23:00:00    0.976000
Name: X26, Length: 8760, dtype: float64

In [31]:
len(test.columns)

200

In [36]:
from before_LSTM import *

In [37]:
before_LSTM(test.iloc[:, 0])

NameError: name 'pd' is not defined